#  LLM based Natural Language to SQL Query Generator

**Description:**

In this project, a system is created, using the OpenAI API and LlamaIndex, that converts natural language queries into SQL queries and executes them on a DuckDB database.

---

## 1. Project Setup and Importing Libraries


### 1.1 Install Required Libraries

In [ ]:
%%capture
!pip install duckdb duckdb-engine llama-index openai llama-index-readers-wikipedia wikipedia

### 1.2 Importing Required Libraries

In [ ]:
# Llama-Index Libraries
from llama_index.core import SQLDatabase, SimpleDirectoryReader, Document
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core.indices.struct_store import (
    NLSQLTableQueryEngine,
    SQLTableRetrieverQueryEngine,
)
# I-Python Library for Displaying in notebook
from IPython.display import Markdown, display

# Text-to-SQL Libraries
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

### 1.3 DuckDB Setup and SQL Engine Setup

In [ ]:
# creates a SQLAlchemy engine object that connects to an in-memory DuckDB database.
engine = create_engine("duckdb:///:memory:") # https://duckdb.org/
metadata_obj = MetaData()

#### 1.3.1 Creating Tables

In [ ]:
# Creating Tables
table_name = "city_statistics"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [ ]:
# print tables
metadata_obj.tables.keys()

dict_keys(['city_stats'])

#### 1.3.2 Test Data

In [ ]:
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000, "country": "United States"},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [ ]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


---

## 2. Creating SQLDatabase Object

In [ ]:
from llama_index.core import SQLDatabase

In [ ]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

/usr/local/lib/python3.10/dist-packages/duckdb_engine/__init__.py:174: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


## 3. Using OpenAI model

### 3.1 Setting up API key

In [ ]:
import os
from google.colab import userdata
# https://platform.openai.com/account/api-keys
os.environ["OPENAI_API_KEY"] = userdata.get("OPEN_API_KEY")

### 3.2 Setting up llama-index to use OpenAI API

In [ ]:
#Configuring Llama Index
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

llm = OpenAI(model="gpt-3.5-turbo")

# configure Settings
Settings.llm = llm

### 3.3 Setting up Query Engine

In [ ]:
query_engine_openai = NLSQLTableQueryEngine(sql_database)



---



## 4. Data Queries

### 4.1 Query 1:

In [ ]:
response = query_engine_openai.query("Which city has the highest population?")

In [ ]:
response.response

'Tokyo has the highest population among all cities, with a population of 13,960,000.'

In [ ]:
response.metadata

{'59620d56-ae2c-40f2-aa10-45904ec08109': {'sql_query': 'SELECT city_name, population\nFROM city_stats\nORDER BY population DESC\nLIMIT 1;',
  'result': [('Tokyo', 13960000)],
  'col_keys': ['city_name', 'population']},
 'sql_query': 'SELECT city_name, population\nFROM city_stats\nORDER BY population DESC\nLIMIT 1;',
 'result': [('Tokyo', 13960000)],
 'col_keys': ['city_name', 'population']}

-----

### 4.2 Query 2:

In [ ]:
response_with_population = query_engine_openai.query("Which city has the highest population. Also provide the population?")

In [ ]:
response_with_population.response

'The city with the highest population is Tokyo, with a population of 13,960,000.'

-------

## 5. Large Database with multiple Tables

In case of a large database with several tables.

### 5.1 Defining Database Tables

In [ ]:
# creates a SQLAlchemy engine object that connects to an in-memory DuckDB database.
engine = create_engine("duckdb:///:memory:")
metadata_obj = MetaData()

In [ ]:
# create city_stats SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
    extend_existing=True,
)

all_table_names = ["city_stats"]

# create a ton of dummy tables
n = 100
for i in range(n):
    tmp_table_name = f"table_{i}"
    tmp_table = Table(
        tmp_table_name,
        metadata_obj,
        Column(f"field_{i}_1", String(16), primary_key=True),
        Column(f"field_{i}_2", Integer),
        Column(f"field_{i}_3", String(16), nullable=False),
        extend_existing=True,
    )
    all_table_names.append(f"table_{i}")

metadata_obj.create_all(engine)

In [ ]:
all_table_names

In [ ]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[]


In [ ]:
# insert dummy data
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000, "country": "United States"},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [ ]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


In [ ]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM table_99")
    print(cursor.fetchall())

[]


In [ ]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

/usr/local/lib/python3.10/dist-packages/duckdb_engine/__init__.py:174: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


### 5.2 Construct Object Index

In [ ]:
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.objects import SQLTableNodeMapping, ObjectIndex, SQLTableSchema
from llama_index.core import VectorStoreIndex

In [ ]:
table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = [SQLTableSchema(table_name=table_name) for table_name in all_table_names]

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)

### 5.3 Query Index with `SQLTableRetrieverQueryEngine`


In [ ]:
query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    obj_index.as_retriever(similarity_top_k=1),
)



---



## 4. Data Queries

### 4.1 Query 1:

In [ ]:
response = query_engine.query("Which city has the highest population?")

In [ ]:
response.response

'Tokyo has the highest population among all cities, with a population of 13,960,000.'

In [ ]:
response.metadata

{'773c589a-b95a-45aa-8e3f-a18aede57467': {'sql_query': 'SELECT city_name, population\nFROM city_stats\nORDER BY population DESC\nLIMIT 1;',
  'result': [('Tokyo', 13960000)],
  'col_keys': ['city_name', 'population']},
 'sql_query': 'SELECT city_name, population\nFROM city_stats\nORDER BY population DESC\nLIMIT 1;',
 'result': [('Tokyo', 13960000)],
 'col_keys': ['city_name', 'population']}



---

